In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division
from __future__ import print_function

This c code from _FirAlgs.c_ seems to be ok

    /****************************************************************************
    * fir_basic: This does the basic FIR algorithm: store input sample, calculate  
    * output sample, shift delay line                                           
    *****************************************************************************/
    SAMPLE fir_basic(SAMPLE input, int ntaps, const SAMPLE h[], SAMPLE z[])       
    {
        int ii;
        SAMPLE accum;

        /* store input at the beginning of the delay line */
        z[0] = input;

        /* calc FIR */
        accum = 0;
        for (ii = 0; ii < ntaps; ii++) {
            accum += h[ii] * z[ii];
        }

        /* shift delay line */
        for (ii = ntaps - 2; ii >= 0; ii--) {
            z[ii + 1] = z[ii];
        }

        return accum;
    }


This test code below from the same file has problems:
- z should be the same length as h (i.e., delay line should be same size as array of coefficients)
- the macro definition of IMP_SIZE is commented out and shouldn't be

    define NTAPS 6
    static const SAMPLE h[NTAPS] = { 1.0, 2.0, 3.0, 4.0, 5.0, 6.0 };
    static SAMPLE h2[2 * NTAPS];
    static SAMPLE z[2 * NTAPS];
    #define IMP_SIZE (3 * NTAPS)
    static SAMPLE imp[IMP_SIZE];
    SAMPLE output;
    int ii, state;

    /* make impulse input signal */
    clear(IMP_SIZE, imp);
    imp[5] = 1.0;

    /* create h2 by doubling h */
    for (ii = 0; ii < NTAPS; ii++) {
        h2[ii] = h2[ii + NTAPS] = h[ii];
    }

    /* test FIR algorithms */

    printf("Testing fir_basic:\n   ");
    clear(NTAPS, z);
    for (ii = 0; ii < IMP_SIZE; ii++) {
        output = fir_basic(imp[ii], NTAPS, h, z);
        printf("%3.1lf ", (double) output);
    }   

In [14]:
def fir_basic(new_value):
    z[0] = new_value
    accum = 0
    for i in range(len(h)):
        accum += h[i] * z[i]
    # shift values to right in z buffer
    for i in range(len(h)-2, -1, -1):
        z[i+1] = z[i]
    #print(z, '\n', h)
    return accum

In [15]:
ntaps = 3
#h = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
h = np.linspace(1.0,ntaps, num=ntaps)
z = np.zeros(ntaps)
imp_size = 3*ntaps
imp = np.zeros(imp_size)
imp[1] = 1.0
print(imp)
print(z)
print(h)

[ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.]
[ 1.  2.  3.]


In [16]:
output = np.zeros(imp_size)
for i in range(imp_size):
    output[i] = fir_basic(imp[i])
    print(i, imp[i], output, z)
print('z:', z)
print('  ', output)

0 0.0 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.]
1 1.0 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.] [ 1.  1.  0.]
2 0.0 [ 0.  1.  2.  0.  0.  0.  0.  0.  0.] [ 0.  0.  1.]
3 0.0 [ 0.  1.  2.  3.  0.  0.  0.  0.  0.] [ 0.  0.  0.]
4 0.0 [ 0.  1.  2.  3.  0.  0.  0.  0.  0.] [ 0.  0.  0.]
5 0.0 [ 0.  1.  2.  3.  0.  0.  0.  0.  0.] [ 0.  0.  0.]
6 0.0 [ 0.  1.  2.  3.  0.  0.  0.  0.  0.] [ 0.  0.  0.]
7 0.0 [ 0.  1.  2.  3.  0.  0.  0.  0.  0.] [ 0.  0.  0.]
8 0.0 [ 0.  1.  2.  3.  0.  0.  0.  0.  0.] [ 0.  0.  0.]
z: [ 0.  0.  0.]
   [ 0.  1.  2.  3.  0.  0.  0.  0.  0.]


Array for delay line, z, should have the same length as the array of filter coefficients, h.